In [1]:
# Two Class Classification of Skin_Cancer Dataset
# a. Use  a  pretrained  VGG-16  model  on  Imagenet  dataset  by  removing  the  top  fully connected layers and adding three dense layers having 64, 32 and 2 neurons with relu, sigmoid and softmax activation functions respectively for classifying the two class Skin_Cancer RGB dataset given in Teams.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob as gb
import cv2
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping ,ReduceLROnPlateau 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , Dense , Dropout , Flatten , MaxPooling2D , BatchNormalization ,experimental
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow import keras
from keras.models import Model

In [3]:
pwd

'c:\\Users\\ashuo\\Downloads\\GithubProjects\\OpenCV-Computer-Vision\\LAB_6'

In [4]:
#data path
trainpath = 'c:\\Users\\ashuo\\Downloads\\GithubProjects\\OpenCV-Computer-Vision\\LAB_6\\Skin_Cancer\\Train'#
testpath = 'c:\\Users\\ashuo\\Downloads\\GithubProjects\\OpenCV-Computer-Vision\\LAB_6\\Skin_Cancer\\Test'

In [5]:
new_size=224
train_images=[]
train_labels=[]
for i in os.listdir(trainpath):#entering train folder
  print("Entering to the folder name:",i)
  files=gb.glob(pathname=str(trainpath+'/' + i + '/*.jpg'))# pointing to all the .jpg extension image folder
  print("Number of images in the folder is",len(files))
  for j in files:# reading each images
      class_cancer={'benign':0,'malignant':1}
      image_raw=cv2.imread(j)
      image=cv2.cvtColor(image_raw,cv2.COLOR_BGR2RGB)
      resize_image=cv2.resize(image,(new_size,new_size))
      train_images.append(list(resize_image))
      train_labels.append(class_cancer[i])

Entering to the folder name: benign
Number of images in the folder is 25
Entering to the folder name: malignant
Number of images in the folder is 25


In [6]:
new_size=224
test_images=[]
test_labels=[]
for i in os.listdir(testpath):# entering to the test folder
  print("Entering to the folder name:",i)
  files=gb.glob(pathname=str(testpath +'/' + i + '/*.jpg'))# pointing to all the .jpg extension image folder
  print("Number of images in the folder is",len(files))
  for j in files:
      class_cancer={'benign':0,'malignant':1}
      image_raw=cv2.imread(j)
      image=cv2.cvtColor(image_raw,cv2.COLOR_BGR2RGB)
      resize_image=cv2.resize(image,(new_size,new_size))
      test_images.append(list(resize_image))
      test_labels.append(class_cancer[i])

Entering to the folder name: benign
Number of images in the folder is 25
Entering to the folder name: malignant
Number of images in the folder is 25


In [7]:
def list_to_array_train(train_images,train_labels):
  return np.array(train_images),np.array(train_labels)

X_train,y_train=list_to_array_train(train_images,train_labels)


def list_to_array_test(test_images,test_labels):
  return np.array(test_images),np.array(test_labels)


X_test,y_test=list_to_array_test(test_images,test_labels)

In [8]:
print(X_train.shape)
print("*"*20)
print(y_train.shape)
print("*"*20)
print(X_test.shape)
print(y_test.shape)

(50, 224, 224, 3)
********************
(50,)
********************
(50, 224, 224, 3)
(50,)


In [9]:
def keras_to_categorical(y_train,y_test):
  return to_categorical(y_train),to_categorical(y_test)

y_train,y_test=keras_to_categorical(y_train,y_test)

In [10]:
def model_vgg16():
  VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224, 3))
  #Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
  for layer in VGG_model.layers:
    layer.trainable = False #True for actual transfer learning
  feature=keras.layers.GlobalAveragePooling2D()(VGG_model.output)
  d1=Dense(units=64,kernel_initializer="glorot_uniform", activation='relu')(feature)
  d2=Dense(units=32,kernel_initializer="glorot_uniform", activation='sigmoid')(d1)
  d3=Dense(units=2,kernel_initializer="glorot_uniform", activation='softmax')(d2)
  output = Model(inputs =VGG_model.input, outputs =d3)
  
  return output

model16=model_vgg16()

In [11]:
model16.compile(optimizer='Adam', loss='mse', metrics='accuracy')

In [12]:
history = model16.fit(X_train, y_train, validation_split=0.2,epochs= 5, batch_size= 5, verbose=1,validation_data=(X_test,y_test))

Epoch 1/5


10/10 [==============================] - 10s 974ms/step - loss: 0.2159 - accuracy: 0.6600 - val_loss: 0.1973 - val_accuracy: 0.7200
Epoch 2/5
10/10 [==============================] - 26s 3s/step - loss: 0.1352 - accuracy: 0.8400 - val_loss: 0.1729 - val_accuracy: 0.7200
Epoch 3/5
10/10 [==============================] - 26s 3s/step - loss: 0.1051 - accuracy: 0.9000 - val_loss: 0.1555 - val_accuracy: 0.7400
Epoch 4/5
10/10 [==============================] - 27s 3s/step - loss: 0.0772 - accuracy: 0.9200 - val_loss: 0.1663 - val_accuracy: 0.8000
Epoch 5/5
10/10 [==============================] - 9s 943ms/step - loss: 0.0581 - accuracy: 0.9600 - val_loss: 0.1431 - val_accuracy: 0.8200


# Feature Extraction

In [13]:
def model_vgg16():
  VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224, 3))
  #Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
  for layer in VGG_model.layers:
    layer.trainable = False #True for actual transfer learning
  feature=keras.layers.GlobalAveragePooling2D()(VGG_model.output)
  
  output = Model(inputs =VGG_model.input, outputs =feature)
  
  return output

model_FE_16=model_vgg16()

# This code essentially creates a modified VGG16 model where the fully connected layers are removed, and the output of the last convolutional layer is followed by a global average pooling layer. This modified model is then compiled and used to extract features from the training and testing datasets (X_train and X_test, respectively). These features can then be used as inputs to other machine learning models or for further analysis.

In [14]:
model_FE_16.compile(optimizer='Adam', loss='mse', metrics='accuracy')
train_feature_16=model_FE_16.predict(X_train)
test_feature_16=model_FE_16.predict(X_test)

2/2 [==============================] - 4s 1s/step


In [15]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf=rf.fit(train_feature_16,y_train)
train_pred=rf.predict(train_feature_16)
test_pred=rf.predict(test_feature_16)
print("Train Accuracy Score",accuracy_score(train_pred,y_train))
print("Test Accuracy Score",accuracy_score(test_pred,y_test))

Train Accuracy Score 1.0
Test Accuracy Score 0.84


In [16]:
def model_vgg19():
  VGG_model = VGG19(weights='imagenet', include_top=False, input_shape=(224,224, 3))
  #Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
  for layer in VGG_model.layers:
    layer.trainable = False #True for actual transfer learning
  feature=keras.layers.GlobalAveragePooling2D()(VGG_model.output)
  ##d1=Dense(units=256,kernel_initializer="glorot_uniform", W_regularizer=l2(0.001), activation='relu')(feature)
  ##d2=Dense(units=2,kernel_initializer="glorot_uniform", W_regularizer=l2(0.001), activation='softmax')(d1)
  #output = Model(inputs =VGG_model.input, outputs =d2)
  output = Model(inputs =VGG_model.input, outputs =feature)
  
  return output

model_FE_19=model_vgg19()

In [17]:
model_FE_19.compile(optimizer='Adam', loss='mse', metrics='accuracy')
train_feature_19=model_FE_19.predict(X_train)
test_feature_19=model_FE_19.predict(X_test)

2/2 [==============================] - 4s 2s/step


# Fusion of Features

In [18]:
final_train=np.hstack((train_feature_16,train_feature_19))
final_test=np.hstack((test_feature_16,test_feature_19))

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf=rf.fit(final_train,y_train)
train_pred=rf.predict(final_train)
test_pred=rf.predict(final_test)
print("Train Accuracy Score",accuracy_score(train_pred,y_train))
print("Test Accuracy Score",accuracy_score(test_pred,y_test))

Train Accuracy Score 1.0
Test Accuracy Score 0.82


In [22]:
x = 5


train_accuracies = []
test_accuracies = []

for _ in range(5):
     rf=RandomForestClassifier(n_estimators=100)
     rf=rf.fit(final_train,y_train)
     train_pred=rf.predict(final_train)
     test_pred=rf.predict(final_test)
     print("Train Accuracy Score",accuracy_score(train_pred,y_train))
     print("Test Accuracy Score",accuracy_score(test_pred,y_test))
     train_accuracies.append(accuracy_score(train_pred,y_train))
     test_accuracies.append(accuracy_score(test_pred,y_test))


print("Train Accuracy Score",np.mean(train_accuracies))
print("Test Accuracy Score",np.mean(test_accuracies))

Train Accuracy Score 1.0
Test Accuracy Score 0.8
Train Accuracy Score 1.0
Test Accuracy Score 0.8
Train Accuracy Score 1.0
Test Accuracy Score 0.82
Train Accuracy Score 1.0
Test Accuracy Score 0.78
Train Accuracy Score 1.0
Test Accuracy Score 0.82
Train Accuracy Score 1.0
Test Accuracy Score 0.804
